# ENERGY LOAD FORECASTING PROJECT
Comparing Facebook Prophet vs SARIMAX for Hourly Predictions

PROJECT OVERVIEW
================
In this project, I'm building a time series forecasting model to predict 
hourly electricity load (demand) in Italy for 2016. 

Why is this important?
- Energy grid operators need accurate load predictions to manage supply
- Solar generation affects net load (more solar = less grid demand)
- Accurate forecasts prevent blackouts and reduce costs

I'll compare two powerful forecasting methods:
1. Facebook Prophet - Great for handling seasonality automatically
2. SARIMAX - Classical statistical approach with external variables

Let's get started! 🚀

# SECTION 1: ENVIRONMENT SETUP

In [ ]:
print("="*70)
print("SECTION 1: INSTALLING PACKAGES AND SETTING UP ENVIRONMENT")
print("="*70)

In [ ]:
# I'm installing all the packages I'll need for this project
# This might take a minute, so be patient!

import sys
import subprocess

# Install required packages
print("\n📦 Installing required packages...")
packages = [
    'pandas',
    'numpy', 
    'matplotlib',
    'plotly',
    'statsmodels',
    'prophet',
    'pmdarima',  # For auto_arima
    'scikit-learn'
]

for package in packages:
    try:
        __import__(package)
        print(f"✓ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✓ {package} installed successfully")

print("\n✓ All packages ready!")

# SECTION 2: IMPORT LIBRARIES

In [ ]:
print("\n" + "="*70)
print("SECTION 2: IMPORTING LIBRARIES")
print("="*70)

In [ ]:
# I'm importing all the necessary libraries for data manipulation, 
# visualization, and modeling

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')  # Hide warnings for cleaner outputs

In [ ]:
# Time series specific imports
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# Metrics for model evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("✓ All libraries imported successfully!")

# SECTION 3: LOAD DATA

In [ ]:
print("\n" + "="*70)
print("SECTION 3: LOADING THE DATASET")
print("="*70)

In [ ]:
# NOTE: Replace this path with your actual file path
# For Google Colab, you can mount Google Drive like this:
# from google.colab import drive
# drive.mount('/content/drive')
# file_path = '/content/drive/MyDrive/IT_2016_Hourly_Energy.csv'

# For local use or if file is in same directory:
file_path = r"YOUR_FILE_PATH"

print(f"\n📂 Loading data from: {file_path}")

In [ ]:
# I'm loading the dataset and immediately converting the timestamp column 
# to a proper datetime format so Python understands it's a date/time

try:
    df = pd.read_csv(file_path, parse_dates=['utc_timestamp'])
    print("✓ Data loaded successfully!")
except FileNotFoundError:
    print("❌ File not found! Please check the file path.")
    print("Creating sample data for demonstration...")
    
    # Create sample data for demonstration if file not found
    dates = pd.date_range('2016-01-01', '2016-12-31 23:00:00', freq='H')
    np.random.seed(42)
    
    # Create synthetic load data with daily and weekly patterns
    hours = np.array([d.hour for d in dates])
    days = np.array([d.dayofweek for d in dates])
    
    # Base load with trend
    base_load = 30000 + np.linspace(0, 5000, len(dates))
    
    # Daily pattern (higher during day)
    daily_pattern = 8000 * np.sin((hours - 6) * np.pi / 12)
    
    # Weekly pattern (lower on weekends)
    weekly_pattern = -2000 * (days >= 5).astype(int)
    
    # Random noise
    noise = np.random.normal(0, 1000, len(dates))
    
    load = base_load + daily_pattern + weekly_pattern + noise
    load = np.maximum(load, 15000)  # Ensure minimum load
    
    # Solar generation (inverse of load during day)
    solar = np.maximum(0, 5000 * np.sin((hours - 6) * np.pi / 12) + 
                       np.random.normal(0, 500, len(dates)))
    
    df = pd.DataFrame({
        'utc_timestamp': dates,
        'IT_load_new': load,
        'IT_solar_generation': solar
    })
    
    print("✓ Sample data created for demonstration!")
    

In [ ]:
# Set timestamp as index for easier time series operations
df.set_index('utc_timestamp', inplace=True)

In [ ]:
# Display basic information about the dataset
print(f"\n📊 Dataset Information:")
print(f"   -Shape: {df.shape[0]} rows x {df.shape[1]} columns")
print(f"   -Date Range: {df.index.min()} to {df.index.max()}")
print(f"   - Frequency: Hourly")
print(f"\n   First few rows:")
print(df.head())

In [ ]:
print(f"\n   Summary Statistics:")
print(df.describe().T)

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print(f"\n   Missing Values:")
print(missing_values)

if missing_values.sum() > 0:
    print("\n⚠ I found missing values! I'll handle them in the next step.")

# SECTION 4: DATA PREPROCESSING

In [ ]:
print("\n" + "="*70)
print("SECTION 4: DATA PREPROCESSING")
print("="*70)

print("\n🔧 Cleaning and preparing the data...")

In [ ]:
# Handle missing values if they exist
if df.isnull().sum().sum() > 0:
    print("\n   Handling missing values...")
    print("   Strategy: Forward fill then backward fill")
    print("   Why? This preserves the time series pattern better than dropping rows")
    df = df.fillna(method='ffill').fillna(method='bfill')
    print("   ✓ Missing values handled!")
else:
    print("   ✓ No missing values found!")

In [ ]:
missing_values = df.isnull().sum()
print(f"\n   Missing Values:")
print(missing_values)

In [ ]:
# Rename columns for clarity in visualizations
df_original = df.copy()   # Keep original for SARIMAX
df.columns = ['Load_MW', 'Solar_MW']

print(f"\n✓ Data preprocessing complete!")
print(f"   Final dataset shape: {df.shape}")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.dtypes

In [ ]:
type(df.index)

# SECTION 5: EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
print("\n" + "="*70)
print("SECTION 5: EXPLORATORY DATA ANALYSIS")
print("="*70)

print("\n📊 Let's visualize the data to understand patterns...")

### 5.1: Time Series Overview

In [ ]:
print("\n[5.1] Creating Time Series Overview Plot...")

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Electricity Load (Demand) Over Time', 
                    'Solar Generation Over Time'),
    vertical_spacing=0.1
)

# Load plot
fig.add_trace(
    go.Scatter(x=df.index, y=df['Load_MW'], 
               name='Load', 
               line=dict(color='royalblue', width=1)),
    row=1, col=1
)

#Solar plot
fig.add_trace(
    go.Scatter(x=df.index, y=df['Solar_MW'], name='Solar',
              line=dict(color='orange', width=1)),
    row=2, col=1
)

fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_yaxes(title_text="Load (MW)", row=1, col=1)
fig.update_yaxes(title_text="Solar (MW)", row=2, col=1)

fig.update_layout(
    height=600,
    title_text="Italy 2016 - Hourly Energy Data Overview",
    showlegend=True,
    hovermode='x unified'
)

fig.show()

print("✓ Time series overview created!")

### 5.2: Seasonal Decomposition

In [ ]:
print("\n[5.2] Performing Seasonal Decomposition...")
print("This breaks down the time series into: Trend + Seasonal + Residual")

In [ ]:
decomposition = seasonal_decompose(df['Load_MW'], model='additive', period=24)

fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=
    ('Original Load', 'Trend', 'Seasonal (12 Months)', 'Residual'),
    vertical_spacing=0.1
)

# Original
fig.add_trace(go.Scatter(x=df.index, y=df['Load_MW'],
                        line=dict(color='blue', width=1),
                        name='Original'),
             row=1, col=1)

# Trend
fig.add_trace(go.Scatter(x=df.index, y=decomposition.trend, 
                         line=dict(color='red', width=2), name='Trend'),
              row=2, col=1)

# Seasonal
fig.add_trace(go.Scatter(x=df.index, y=decomposition.seasonal, 
                         line=dict(color='green', width=1), name='Seasonal'),
              row=3, col=1)

# Residual
fig.add_trace(go.Scatter(x=df.index, y=decomposition.resid, 
                         line=dict(color='gray', width=1), name='Residual'),
              row=4, col=1)

fig.update_layout(height=800, title_text="Time Series Decomposition", showlegend=False)
fig.show()

print("✓ Decomposition complete!")

### **Key Findings on Electricity Load (`Load_MW` - Demand)**

I noticed that electricity demand is primarily driven by seasonal and calendar effects:

1.  **Dual Seasonality:** The load peaks significantly in two distinct periods, reflecting the use of climate control:
    * **Winter Peaks (Jan, Nov, Dec):** Driven by **heating** demand.
    * **Summer Peaks (Jul):** Driven by **air conditioning** (cooling) demand.
2.  **Calendar Effects (Holiday Dips):** I observed sharp, sudden drops in demand that are critical to note:
    * **Mid-August Dip:** This aligns perfectly with the ***Ferragosto*** national holiday period in Italy, where much of the commercial and industrial load shuts down.
    * **Other Dips:** Smaller but noticeable drops in **April, October, and December** are likely due to other national holidays (like Easter) and the Christmas/New Year's holiday period.

> **➡️ Actionable Insight:** Since these holiday dips are predictable, I will use **Prophet's holiday feature** to explicitly model them. This prevents the model from interpreting them as random noise, which would ruin the forecast for a regular weekday.

---

### **Key Findings on Solar Generation (`Solar_MW` - Exogenous Variable)**

The solar plot confirms its function as a vital external predictor for my model:

1.  **Physical Trend:** The plot shows a clear, inverse U-shaped curve, peaking around the summer months (June/July). This is simply driven by **physics**: the longest, most intense daylight hours occur in the summer.
2.  **Inverse Correlation:** As expected, the Solar Generation is a *counterweight* to the Load. During sunny hours, local generation meets some demand, causing the net demand on the central grid (`Load_MW`) to fall.

> **➡️ Actionable Insight:** I must include the `Solar_MW` column as an **exogenous variable (regressor)** in both the Prophet and SARIMAX models to capture this vital relationship and maximize forecast accuracy.

---

### **Critical Data Anomaly Found** 🚨

I discovered a major issue around the end of October that requires immediate attention:

* **Massive Spike (Oct 29-31):** The Solar Generation plot shows an extreme, unphysical spike. In energy time series, this is the classic signature of an error related to the **Daylight Saving Time (DST)** transition, where an hour is duplicated or miscalculated.

> **➡️ Actionable Insight:** This spike is a powerful outlier. Before fitting either model, I must perform **data cleaning** to **impute or remove** this handful of corrupted data points. Ignoring it would severely skew the parameters of the SARIMAX model and negatively impact the performance of Prophet.



In [ ]:
# Filter the Solar_MW data for October to mid-November 2016
# This window ensures we capture the anomaly and the clean data around it.
solar_oct_nov = df['Solar_MW'].loc['2016-10-01':'2016-11-15']

# Create the Plotly visualization
fig = go.Figure()

# 1. Plot the Solar MW time series
fig.add_trace(go.Scatter(
    x=solar_oct_nov.index,
    y=solar_oct_nov.values,
    mode='lines',
    name='Solar Generation (MW)',
    line=dict(color='orange', width=2)
))

# 2. Add vertical lines for the previous anomaly boundary estimates (for reference)
# These were the estimates (2016-10-29 to 2016-11-01) that might have been slightly off.
max_y = solar_oct_nov.max() if not solar_oct_nov.empty else 10000

fig.update_layout(
    title='Solar Generation (MW) - Anomaly Detection Window (Oct-Nov 2016)',
    xaxis_title='Date',
    yaxis_title='Solar Generation (MW)',
    height=600,
    shapes=[
        # Previous Start Estimate: 2016-10-29
        dict(
            type="line",
            x0='2016-10-29', y0=0, x1='2016-10-29', y1=max_y,
            line=dict(color="Red", width=1, dash="dot"),
        ),
        # Previous End Estimate: 2016-11-01
        dict(
            type="line",
            x0='2016-11-01', y0=0, x1='2016-11-01', y1=max_y,
            line=dict(color="Red", width=1, dash="dot"),
        )
    ],
    annotations=[
        dict(x='2016-10-29', y=max_y * 0.95, xref="x", yref="y", 
             text="Previous Start Estimate", showarrow=False, font=dict(color="Red")),
        dict(x='2016-11-01', y=max_y * 0.90, xref="x", yref="y", 
             text="Previous End Estimate", showarrow=False, font=dict(color="Red"))
    ]
)

fig.show()

In [ ]:
# Define the exact window identified by visual inspection
anomaly_start = '2016-10-27 00:00:00'
anomaly_end = '2016-11-01 00:00:00' 

print("SECTION: FINAL DST Anomaly Repair (Linear Interpolation)")
print("="*60)

# 1. Capture the original, corrupted data for inspection
original_solar = df['Solar_MW'].loc[anomaly_start:anomaly_end].copy()
original_load = df['Load_MW'].loc[anomaly_start:anomaly_end].copy()

# 2. ISOLATION: Temporarily set the outlier values to NaN
df.loc[anomaly_start:anomaly_end, 'Load_MW'] = np.nan
df.loc[anomaly_start:anomaly_end, 'Solar_MW'] = np.nan

# 3. INTERPOLATION (The Fix): Apply the safer 'linear' method directly
# Linear interpolation prevents the "overshoot" that caused negative solar values.
df['Load_MW'] = df['Load_MW'].interpolate(method='linear')
df['Solar_MW'] = df['Solar_MW'].interpolate(method='linear')

# 4. Extract the newly imputed values for verification
imputed_solar = df['Solar_MW'].loc[anomaly_start:anomaly_end]
imputed_load = df['Load_MW'].loc[anomaly_start:anomaly_end]


print(f"✅ Anomaly Window successfully repaired using LINEAR interpolation: {anomaly_start} to {anomaly_end}")
print("-" * 60)

# Create a comparison table for inspection
comparison_df = pd.DataFrame({
    'Original Solar (MW)': original_solar,
    'Imputed Solar (MW)': imputed_solar,
    'Original Load (MW)': original_load,
    'Imputed Load (MW)': imputed_load
}).dropna(subset=['Original Solar (MW)'])

print("\nSample of Imputed Values (Now Linear) vs. Corrupted Values:")
print(comparison_df.head(10).round(2))
print("-" * 60)

In [ ]:
print("\n[5.1] Creating Time Series Overview Plot...")

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Electricity Load (Demand) Over Time', 
                    'Solar Generation Over Time'),
    vertical_spacing=0.1
)

# Load plot
fig.add_trace(
    go.Scatter(x=df.index, y=df['Load_MW'], 
               name='Load', 
               line=dict(color='royalblue', width=1)),
    row=1, col=1
)

#Solar plot
fig.add_trace(
    go.Scatter(x=df.index, y=df['Solar_MW'], name='Solar',
              line=dict(color='orange', width=1)),
    row=2, col=1
)

fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_yaxes(title_text="Load (MW)", row=1, col=1)
fig.update_yaxes(title_text="Solar (MW)", row=2, col=1)

fig.update_layout(
    height=600,
    title_text="Italy 2016 - Hourly Energy Data Overview (After outlier treatment)",
    showlegend=True,
    hovermode='x unified'
)

fig.show()

print("✓ Time series overview created!")

### 5.3: Daily Pattern Heatmap

In [ ]:
print("\n[5.3] Creating Daily Pattern Heatmap...")
print("This shows average load by hour and day of week")

In [ ]:
# Create hour and day columns
df_heatmap = df.copy()  # I'm creating a copy of the main data so I don't accidentally change the original.
df_heatmap['Hour'] = df_heatmap.index.hour
df_heatmap['DayOfWeek'] = df_heatmap.index.dayofweek
df_heatmap['DayName'] = df_heatmap.index.day_name()

In [ ]:
df_heatmap

In [ ]:
# Calculate the average load by hour and day
pivot_table = df_heatmap.pivot_table(
    values='Load_MW',
    index='DayName',
    columns='Hour',
    aggfunc='mean'
)

In [ ]:
pivot_table.T

In [ ]:
# Reorder days correctly
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
pivot_table = pivot_table.reindex(day_order)

In [ ]:
fig = go.Figure(data=go.Heatmap(
    z=pivot_table.values,
    x=pivot_table.columns,
    y=pivot_table.index,
    colorscale='RdYlBu_r',
    colorbar=dict(title="Load (MW)")
))

fig.update_layout(
    title='Average Hourly Load Pattern by Day of Week',
    xaxis_title='Hour of Day',
    yaxis_title='Day of Week',
    height=500
)

fig.show()

print("✓ Heatmap created!")

### 5.4: Load vs Solar Relationship

In [ ]:
print("\n[5.4] Analyzing Load vs Solar Generation Relationship...")

In [ ]:
# Create a temporary DataFrame for normalized values
df_norm = df[['Load_MW', 'Solar_MW']].copy()

# Normalize the data between 0 and 1 (Min-Max Scaling)
# This allows us to plot them on the same vertical scale for comparison
for col in df_norm.columns:
    df_norm[col] = (df_norm[col] - df_norm[col].min()) / (df_norm[col].max() - df_norm[col].min())

# Create the plot using normalized data
fig = go.Figure()

# Load Trace (Blue)
fig.add_trace(go.Scatter(
    x=df_norm.index, y=df_norm['Load_MW'],
    name='Load (Normalized)',
    line=dict(color='royalblue', width=1)
))

# Solar Trace (Orange) - Inverted to show the direct opposite trend
# We invert the solar data (1 - value) to show how it negatively impacts the load.
fig.add_trace(go.Scatter(
    x=df_norm.index, y=1 - df_norm['Solar_MW'],
    name='Solar (Inverted & Normalized)',
    line=dict(color='orange', width=1, dash='dot')
))

fig.update_layout(
    title='Load vs. Inverted Solar (Normalized)',
    yaxis_title='Normalized Scale (0-1)',
    xaxis_title='Date',
    height=500,
    hovermode='x unified'
)

fig.show()

print("\nInsight:")
print("The lines mirror each other, clearly showing the inverse relationship. When the inverted solar line (orange) is high, the load line (blue) is also high, demonstrating that when *actual* solar is low, load is high.")

# SECTION 6: TRAIN-TEST SPLIT

In [ ]:
print("\n" + "="*70)
print("SECTION 6: SPLITTING DATA INTO TRAIN AND TEST SETS")
print("="*70)

In [ ]:
print("\n✂️ Splitting data for model evaluation...")
print("Strategy: Use last 30 days (720 hours) for testing")
print("Why? This represents a realistic forecasting horizon for energy planning")

In [ ]:
# Calculate split point (last 30 days = 720 hours)
test_size = 30 * 24  # 30 days x 24 hours
split_index = len(df) - test_size

In [ ]:
# Split the data
train_df = df.iloc[:split_index]
test_df = df.iloc[split_index:]

In [ ]:
print(f"\n📊 Split Summary:")
print(f"   Training Set: {len(train_df)} hours ({len(train_df)//24} days)")
print(f"   Test Set: {len(test_df)} hours ({len(test_df)//24} days)")
print(f"   Train Period: {train_df.index.min()} to {train_df.index.max()}")
print(f"   Test Period: {test_df.index.min()} to {test_df.index.max()}")

In [ ]:
# Visualize the split
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=train_df.index, y=train_df['Load_MW'],
    name='Training Data', line=dict(color='blue', width=1)))

fig.add_trace(go.Scatter(
    x=test_df.index, y=test_df['Load_MW'],
    name='Test Data', line=dict(color='red', width=1)
))

fig.update_layout(
    title='Train-Test Split Visualization',
    height=800,
    xaxis_title='Date',
    yaxis_title='Load (MW)'
)

fig.show()

print("✓ Data split complete!")

# SECTION 7: FACEBOOK PROPHET MODEL

🔮 What is Prophet?
   - Developed by Facebook (Meta) for forecasting at scale
   - Automatically handles multiple seasonalities (daily, weekly, yearly)
   - Great for business time series with strong seasonal patterns
   - Can include external regressors (like solar generation!)

### 7.1: Prepare Data for Prophet

Preparing data for Prophet...

Prophet requires specific column names: 'ds' (date) and 'y' (target)

In [ ]:
# Create Prophet dataframe
prophet_train = pd.DataFrame({
    'ds': train_df.index,
    'y': train_df['Load_MW'].values,
    'solar': train_df['Solar_MW']   # External regressor
})

In [ ]:
prophet_train.head()

In [ ]:
prophet_test = pd.DataFrame({
    'ds': test_df.index,
    'solar': test_df['Solar_MW'].values  # Need solar for predictions
})

In [ ]:
print("✓ Data prepared!")
print(f"   Training samples: {len(prophet_train)}")
print(f"   Test samples: {len(prophet_test)}")

### 7.2: Configure and Train Prophet

Configuring and training Prophet model...

Configuration:
   - Daily seasonality: ON (24-hour patterns)
   - Weekly seasonality: ON (weekday vs weekend)
   - Yearly seasonality: ON (seasonal changes)
   - Adding 'solar' as external regressor

In [ ]:
# Initialize Prophet with configurations
# prophet_model = Prophet(
#     daily_seasonality=True,
#     weekly_seasonality=True,
#     yearly_seasonality=True,
#     seasonality_mode='additive',
#     interval_width=0.95
# )

prophet_model = Prophet()

# Add solar generation as external regressor
prophet_model.add_regressor('solar')

# Automatically include italian national holidays
prophet_model.add_country_holidays(country_name='IT')
print("   - Added national holidays for Italy ('IT')")

In [ ]:
# Before fitting, ensure the 'ds' column is timezone-naive
# This is a mandatory step for Prophet if your data has timezone info
if prophet_train['ds'].dt.tz is not None:
    prophet_train['ds'] = prophet_train['ds'].dt.tz_localize(None)
    print("✅ Removed timezone from 'ds' column.")

In [ ]:
prophet_train.head()

In [ ]:
print("\n⏳ Training Prophet model... (this may take a minute)")
# Fit the model
prophet_model.fit(prophet_train)
print("✓ Prophet model trained successfully!")

### 7.3: Make Predictions

Generating Prophet forecasts...

NOTE: You MUST also remove the timezone from the future/test dataframe!

In [ ]:
future = prophet_test.copy()
if future['ds'].dt.tz is not None:
    future['ds'] = future['ds'].dt.tz_localize(None)

In [ ]:
# Make predictions
prophet_forecast = prophet_model.predict(future)
print("✓ Forecasts generated!")
print(f"   Forecast columns available: {list(prophet_forecast.columns)}")

### 7.4: Visualize Prophet Results

In [ ]:
fig = go.Figure()

# Actual test data
fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df['Load_MW'],
    name='Actual Load',
    line=dict(color='black', width=2)
))

# Prophet prediction
fig.add_trace(go.Scatter(
    x=prophet_forecast['ds'],
    y=prophet_forecast['yhat'],
    name='Prophet Forecast',
    line=dict(color='blue', width=2, dash='dash')
))

# Confidence interval
fig.add_trace(go.Scatter(
    x=prophet_forecast['ds'],
    y=prophet_forecast['yhat_upper'],
    fill=None,
    mode='lines',
    line=dict(color='lightblue', width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=prophet_forecast['ds'],
    y=prophet_forecast['yhat_lower'],
    fill='tonexty',
    mode='lines',
    line=dict(color='lightblue', width=0),
    name='95% Confidence Interval'
))

fig.update_layout(
    title='Prophet Forecast vs Actual Load (30-Day Test Period)',
    xaxis_title='Date',
    yaxis_title='Load (MW)',
    hovermode='x unified',
    height=500
)

fig.show()

print("✓ Visualization complete!")

### 7.5: Prophet Components

Analyzing Prophet components...

This shows how each factor contributes to the forecast

In [ ]:
# Plot components
fig_components = prophet_model.plot_components(prophet_forecast)
plt.tight_layout()
plt.show()

# SECTION 8: SARIMAX MODEL

📈 What is SARIMAX?
   - Seasonal AutoRegressive Integrated Moving Average with eXogenous variables
   - Classical statistical approach for time series
   - Explicitly models seasonality with seasonal parameters
   - Can include external variables (exogenous regressors)

### 8.1: Find Optimal Parameters with auto_arima

Finding optimal SARIMAX parameters...

I'm using auto_arima to automatically find the best parameters

This tests many combinations and picks the best based on AIC

⏳ This might take several minutes... grab a coffee! ☕

In [ ]:
# # Use auto_arima to find optimal parameters
# # I'm limiting the search space to keep it manageable for beginners (myself)

# stepwise_model = auto_arima(
#     train_df['Load_MW'],
#     exogenous=train_df[['Solar_MW']],
#     seasonal=True,
#     m=24,  # Seasonal period (24 hours)
#     max_p=3,  # Max AR order
#     max_q=3,  # Max MA order
#     max_P=2,  # Max seasonal AR
#     max_Q=2,  # Max seasonal MA
#     max_d=1,  # Max differencing
#     max_D=1,  # Max seasonal differencing
#     trace=True,  # Print progress
#     error_action='ignore',
#     suppress_warnings=True,
#     stepwise=True,  # Use stepwise algorithm (faster)
# )

# print("\n✓ Optimal parameters found!")
# print(f"\nBest Model: {stepwise_model.order} × {stepwise_model.seasonal_order}")
# print(f"AIC: {stepwise_model.aic():.2f}")

### 8.2: Train SARIMAX with Optimal Parameters

Training SARIMAX model with optimal parameters...

In [ ]:
# # Build SARIMAX model
# sarimax_model = SARIMAX(
#     train_df['Load_MW'],
#     exog=train_df[['Solar_MW']],
#     order=stepwise_model.order,
#     seasonal_order=stepwise_model.seasonal_order,
#     enforce_stationarity=False,
#     enforce_invertibility=False
# )

In [ ]:
# Build SARIMAX model
sarimax_model = SARIMAX(
    train_df['Load_MW'],
    exog=train_df[['Solar_MW']],
    # Hardcoding the optimal non-seasonal order (p, d, q)
    order=(2, 1, 1),
    # Hardcoding the optimal seasonal order (P, D, Q, m)
    seasonal_order=(1, 0, 1, 24),
)

# NOTE: Since the parameters are hardcoded, you can skip the 'stepwise_model.fit()' 
# call and move directly to fitting the sarimax_model:
# sarimax_results = sarimax_model.fit(disp=False)

In [ ]:
# Fit the model
print("⏳ Fitting SARIMAX model...")
sarimax_fitted = sarimax_model.fit(disp=False)

In [ ]:
print("✓ SARIMAX model trained successfully!")
print("\nModel Summary:")
print(sarimax_fitted.summary())

### 8.3: Make SARIMAX Predictions

In [ ]:
# Generate predictions
sarimax_forecast = sarimax_fitted.forecast(
    steps=len(test_df),
    exog=test_df[['Solar_MW']]
)

print("✓ SARIMAX forecasts generated!")

In [ ]:
sarimax_forecast

### 8.4: Visualize SARIMAX Results

In [ ]:
fig = go.Figure()

# Actual test data
fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df['Load_MW'],
    name='Actual Load',
    line=dict(color='black', width=2)
))

# SARIMAX prediction
fig.add_trace(go.Scatter(
    x=test_df.index,
    y=sarimax_forecast,
    name='SARIMAX Forecast',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title='SARIMAX Forecast vs Actual Load (30-Day Test Period)',
    xaxis_title='Date',
    yaxis_title='Load (MW)',
    hovermode='x unified',
    height=500
)

fig.show()

print("✓ SARIMAX visualization complete!")

# SECTION 9: MODEL COMPARISON AND EVALUATION

COMPARING PROPHET VS SARIMAX

In [ ]:
# Extract predictions
prophet_pred = prophet_forecast['yhat'].values
sarimax_pred = sarimax_forecast.values
actual = test_df['Load_MW'].values

In [ ]:
# Calculate metrics
def calculate_metrics(actual, predicted, model_name):
    """Calculate and display performance metrics"""
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    print(f"\n{model_name} Performance:")
    print(f"   MAE (Mean Absolute Error): {mae:.2f} MW")
    print(f"   RMSE (Root Mean Squared Error): {rmse:.2f} MW")
    print(f"   MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
    
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

In [ ]:
# Calculate metrics for both models
prophet_metrics = calculate_metrics(actual, prophet_pred, "PROPHET")
sarimax_metrics = calculate_metrics(actual, sarimax_pred, "SARIMAX")

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Prophet': prophet_metrics,
    'SARIMAX': sarimax_metrics
})

In [ ]:
# Determine winner
if prophet_metrics['MAPE'] < sarimax_metrics['MAPE']:
    winner = "Prophet"
    difference = sarimax_metrics['MAPE'] - prophet_metrics['MAPE']
else:
    winner = "SARIMAX"
    difference = prophet_metrics['MAPE'] - sarimax_metrics['MAPE']

print(f"\n🏆 WINNER: {winner}")
print(f"   Better by {difference:.2f}% MAPE")

### 9.1: Side-by-Side Comparison Plot

In [ ]:
fig = go.Figure()

# Actual data
fig.add_trace(go.Scatter(
    x=test_df.index,
    y=actual,
    name='Actual Load',
    line=dict(color='black', width=2.5)
))

# Prophet forecast
fig.add_trace(go.Scatter(
    x=test_df.index,
    y=prophet_pred,
    name='Prophet Forecast',
    line=dict(color='blue', width=2, dash='dash')
))

# SARIMAX forecast
fig.add_trace(go.Scatter(
    x=test_df.index,
    y=sarimax_pred,
    name='SARIMAX Forecast',
    line=dict(color='red', width=2, dash='dot')
))

fig.update_layout(
    title='Model Comparison: Prophet vs SARIMAX vs Actual Load',
    xaxis_title='Date',
    yaxis_title='Load (MW)',
    hovermode='x unified',
    height=600,
    legend=dict(x=0.01, y=0.99)
)

fig.show()

print("✓ Comparison visualization complete!")

### 9.2 Error Analysis

In [ ]:
# Calculate errors
prophet_errors = actual - prophet_pred
sarimax_errors = actual - sarimax_pred

# Create error distribution plot
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Prophet Errors', 'SARIMAX Errors')
)

fig.add_trace(
    go.Histogram(x=prophet_errors, name='Prophet', 
                 marker_color='blue', nbinsx=50),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=sarimax_errors, name='SARIMAX', 
                 marker_color='red', nbinsx=50),
    row=1, col=2
)

fig.update_xaxes(title_text="Error (MW)", row=1, col=1)
fig.update_xaxes(title_text="Error (MW)", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=1, col=1)

fig.update_layout(
    title_text="Error Distribution Analysis",
    showlegend=False,
    height=400
)

fig.show()

print("✓ Error analysis complete!")
print("\nError Insights:")
print(f"   Prophet Error Std Dev: {np.std(prophet_errors):.2f} MW")
print(f"   SARIMAX Error Std Dev: {np.std(sarimax_errors):.2f} MW")
print(f"   Prophet Error Mean: {np.mean(prophet_errors):.2f} MW")
print(f"   SARIMAX Error Mean: {np.mean(sarimax_errors):.2f} MW")